```{glue:} glued_fig
```
# Cell nuclei detection using StarDist

In this notebook, we will use StarDist to detect cell nuclei in an image extracted from a public dataset of histopathology images.

[StarDist](https://github.com/stardist/stardist) is a deep-learning based Python library used for segmenting star-convex objects, such as cell nuclei, in 2D and 3D images. It is also available as plugins for [ImageJ](https://imagej.net/plugins/stardist), [Napari](https://github.com/stardist/stardist-napari), and [Qupath](https://qupath.readthedocs.io/en/0.3/docs/advanced/stardist.html).


In [ ]:
from myst_nb import glue  # To paste the figure output at the top of the notebook

## Setup

Check that you have the `stardist` package installed (`pip install stardist`).

In [ ]:
from stardist.models import StarDist2D

## Download and read the image

The image we'll use in this tutorial is available for download on [Zenodo](https://zenodo.org/record/8099852) (`deepslide.png`). This image is part of the [DeepSlides](https://zenodo.org/record/1184621) public dataset.

In the cell below, we download this image from Zenodo and read it into a Numpy array.

In [ ]:
from shared_data import DATASET  # This dataset has a reference to the image on Zenodo.
from skimage.io import imread

image_file = DATASET.fetch("deepslide.png")

image = imread(image_file)

print(f'Loaded image in an array of shape: {image.shape} and data type {image.dtype}')
print(f'Intensity range: [{image.min()} - {image.max()}]')

## Visualize the image using matplotlib

Let's have a quick look at our example image.

In [ ]:
import matplotlib.pyplot as plt

fig_mpl, ax = plt.subplots(figsize=(8, 8))
ax.imshow(image, cmap='gray')
plt.title("H&E (DeepSlides)")
plt.axis('off')  # Hide axes ticks
plt.show()

## Normalize the image intensity

Let's rescale our image to the range 0-1. By doing so, it is also converted to an array of data type `float`.

In [ ]:
from skimage.exposure import rescale_intensity

image_normed = rescale_intensity(image, out_range=(0, 1))

print(f'Intensity range: [{image_normed.min()} - {image_normed.max()}]')
print(f'Array type: {image_normed.dtype}')

## Initialize a pre-trained StarDist model

The StarDist developers provide a few pre-trained models that may already be applied to suitable images.

Here, we will use the *Versatile (H&E nuclei)* model that was trained on images from the MoNuSeg 2018 training data and the TNBC dataset from Naylor et al. (2018).

In [ ]:
model = StarDist2D.from_pretrained("2D_versatile_he")

model

## Run the model

We use the `predict_instances` method of the model to generate a segmenation mask (`labels`) and a representation of the cell nuclei as polygons (`polys`).

In [ ]:
labels, polys = model.predict_instances(
    image_normed,  # The image must be normalized
    axes="YXC",
    prob_thresh=0.5,  # Detection probability threshold
    nms_thresh=0.1,  # Remove detections overlapping by more than this threshold
    scale=1,  # Higher values are suitable for lower resolution data
    return_labels=True,
)

# We also get detection probabilities
probabilities = list(polys["prob"])

n_detections = len(probabilities)

print(f'{n_detections} cells detected.')

## Visualize the segmentation using `label2rgb`

The `label2rgb` function from Scikit-image assigns a random color to each object in a labeled segmentation mask. This is useful for visualization!

In [ ]:
from skimage.color import label2rgb

fig, ax = plt.subplots(figsize=(12, 6))
rgb_composite = label2rgb(labels, image=image, bg_label=0)
ax.imshow(rgb_composite)
plt.axis('off')
plt.show()

## Visualize the segmentation using `Pillow`

In the cell below, we use the `Pillow` library to overlay a segmentation mask on top of the image. The shade of green represents the detection probability.

In [ ]:
from PIL import Image, ImageOps
import numpy as np

probabilities.insert(0, 0)

parametric_image = np.take(probabilities, labels)

image_ = Image.fromarray(image).convert("RGBA")

mask = Image.fromarray((labels == 0).astype('uint8')).convert("L")

alpha_mask = mask.point(lambda p: 128 if p == 0 else 0)

colorized_mask = ImageOps.colorize(
    Image.fromarray((parametric_image * 255).astype(np.uint8)).convert("L"), 
    black="green", 
    white="yellow", 
    blackpoint=128,
    whitepoint=255,
).convert("RGBA")

colorized_mask.putalpha(alpha_mask)

composite_image = Image.alpha_composite(image_, colorized_mask)

In [ ]:
glue('glued_fig', composite_image, display=False);

```{glue:} glued_fig
```

## Conclusion

In this notebook, we have used `StarDist` to segment cell nuclei in an image from the DeepSlides dataset.